In [ ]:
from sentinelsat import SentinelAPI

api = SentinelAPI('michogar', 'vayamierdadeclave')

In [ ]:
import geopandas as gpd

bbox = gpd.read_file('bbox.geojson')

bbox.explore()

In [ ]:
from sentinelsat import read_geojson, geojson_to_wkt

footprint = geojson_to_wkt(read_geojson('bbox.geojson'))

products = api.query(footprint,
                     date=('20220101', '20220518'),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=0)

products_df = api.to_geodataframe(products)
products_df.to_file("products.gpkg", layer='footprints', driver="GPKG")

products_df.iloc[0]

In [ ]:
uuid = '293dfd0f-b056-447e-8a49-77301ddea3e1'
#api.download(uuid)

In [ ]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# NDVI (Sentinel 2) = (B8 – B4) / (B8 + B4)
# B4	10 m	665 nm	Red
# B8	10 m	842 nm	Visible and Near Infrared (VNIR)

s2s_path = './S2B_MSIL1C_20220318T113319_N0400_R080_T29TNG_20220318T120828.SAFE/GRANULE/L1C_T29TNG_A026275_20220318T113316/IMG_DATA/'

base_name = 'T29TNG_20220318T113319'
b8 = f'{base_name}_B08.jp2'
b4 = f'{base_name}_B04.jp2'

b8_path = f'{s2s_path}/{b8}'
b4_path = f'{s2s_path}/{b4}'

with rasterio.open(b8_path) as src:
   oviews = src.overviews(1)
   oview = oviews[-1]
   thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))
   pixel_size = src.transform[0]
   print(f'Num of bands: {src.count}')
   print(f'Size in Pixels: {src.width} x {src.height} ')
   print(f'Pixel: {pixel_size} m')
   print(f'Size in Km: {src.width * pixel_size / 1000} x {src.height * pixel_size / 1000} ')
   print(f'CRS: {src.crs} ')
   print(f'Origin: {src.transform[2]} , {src.transform[5]}')

plt.imshow(thumbnail, cmap='gray')
plt.colorbar()
plt.title('Overview - Band 8 {}'.format(thumbnail.shape))

In [ ]:
import fiona
import pyproj
from shapely.geometry import shape
from shapely.ops import transform

wgs84 = pyproj.CRS('EPSG:4326')
utm = pyproj.CRS('EPSG:32629')

project = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform

# with fiona.open("bbox.geojson", "r") as bbox_geojson:
#     shapes = [feature["geometry"] for feature in bbox_geojson]
#
# print(shapes[0])

with fiona.open("bbox.geojson", "r") as bbox_geojson:
    shapes = [transform(project, shape(feature["geometry"])) for feature in bbox_geojson]

print(shapes[0].wkt)

EPSG:4326
```
{
    "type": "Polygon",
  "coordinates":
    [[(-8.945285297162739, 41.94764885739232),
      (-8.945285297162739, 42.34741439813461),
      (-8.31885518969755, 42.34741439813461),
      (-8.31885518969755, 41.94764885739232),
      (-8.945285297162739, 41.94764885739232)]]
}
```

EPSG:32629

```
POLYGON ((504535.0538171259 4643965.211482396, 504506.610334198 4688351.868811505, 556103.0375257639 4688575.066893593, 556457.1504084462 4644188.093542257, 504535.0538171259 4643965.211482396))
```

In [ ]:
import rasterio.mask

NODATA = 0
s2_bands = [
    b8_path,
    b4_path
]

for band in s2_bands:
    with rasterio.open(band) as src:
        mask_image, mask_transform = rasterio.mask.mask(src,
                                                      shapes,
                                                      crop=True,
                                                      all_touched=True,
                                                      nodata=NODATA)
        out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                     "height": mask_image.shape[1],
                     "width": mask_image.shape[2],
                     "transform": mask_transform,
                     "nodata": NODATA
                     })

    with rasterio.open(f"{band}.masked.tif", "w", **out_meta) as dest:
        dest.write(mask_image)

In [ ]:
BAND = 1
arrs = []

for band in s2_bands:
    print(f'Reading bands from: {band}.masked.tif')
    with rasterio.open(f'{band}.masked.tif') as f:
        arrs.append(f.read(BAND))

B8 = 0
B4 = 1

sentinel_b8 = np.array(arrs[B8], dtype=arrs[B8].dtype).astype(float)
sentinel_b4 = np.array(arrs[B4], dtype=arrs[B4].dtype).astype(float)

print(sentinel_b8.shape)
print(sentinel_b8[:100,:100])
print(sentinel_b4)

In [ ]:
np.seterr(divide='ignore', invalid='ignore')
ndvi = (sentinel_b8 - sentinel_b4) / (sentinel_b8 + sentinel_b4)

ndvi[np.isnan(ndvi)] = 0

print('Max NDVI: {m}'.format(m=ndvi.max()))
print('Mean NDVI: {m}'.format(m=ndvi.mean()))
print('Median NDVI: {m}'.format(m=np.median(ndvi)))
print('Min NDVI: {m}'.format(m=ndvi.min()))

plt.imshow(ndvi, cmap='RdYlGn')
plt.colorbar()
plt.title('NDVI')


In [ ]:
with rasterio.open(f"{s2_bands[B8]}.masked.tif") as src:
    profile = src.profile.copy()

    profile.update({
            'dtype': 'float32',
            'height': ndvi.shape[0],
            'width': ndvi.shape[1],
    })

    with rasterio.open('ndvi.tif', 'w', **profile) as dst:
        dst.write_band(BAND, ndvi)
